In [1]:
!git clone https://github.com/ngoxuanphong/ENV.git

Cloning into 'ENV'...
remote: Enumerating objects: 1225, done.
remote: Counting objects: 100% (451/451), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 1225 (delta 173), reused 373 (delta 119), pack-reused 774
Receiving objects: 100% (1225/1225), 58.04 MiB | 16.86 MiB/s, done.
Resolving deltas: 100% (483/483), done.


In [2]:
%cd ENV

/content/ENV


In [3]:

# Check hệ thống

# import Base.MachiKoro.env as env
# from CheckEnv import check_env
# print(check_env(env))

import numpy as np
import random as rd
from numba import njit, jit
import sys, os
from setup import SHORT_PATH
import importlib.util
game_name = 'Durak'

In [4]:

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2


In [6]:
per =np.zeros((0.0))

In [21]:
@njit()
def SuperStupidBot(state, per):

    # print(per)


    ValidAction = getValidActions(state)
    ValidAction = np.where(ValidAction ==1)[0]
    returnAction = -1

    trumpSuit = np.where(state[158:162] == 1)[0][0]
    

    for i in range(13):
        for j in range(4):
            if i + 13*j in ValidAction and j != trumpSuit:
                  returnAction = i + 13* j
                  break
        if returnAction != -1:
            break
    if state[157] == 1:
        attackCard = np.where(state[104:156] == 1)[0][0]
        if attackCard // 13 == trumpSuit and np.sum(state[52:104]) <= 4 and np.min(state[163:166]) >= 3:
            returnAction = 52
        elif attackCard % 13 >= 7 and np.sum(state[52:104]) <= 6 and state[162] > 0:
            returnAction = 52
        elif state[162] > 0:
            returnAction = -1
            for i in range(7,13):
                    for j in range(4):
                        if i + 13*j in ValidAction and j != trumpSuit:
                              returnAction = i + 13* j
                              break
                    if returnAction != -1:
                        break
            
            if returnAction == -1:
                for i in range(7):
                    for j in range(4):
                        if i + 13*j in ValidAction and j != trumpSuit:
                              returnAction = i + 13* j
                              break
                    if returnAction != -1:
                        break


    if state[156] == 1:
        if np.sum(state[52:104]) > 0:
            if returnAction != 52:
                if (returnAction % 13 >= 11 and returnAction // 13 == trumpSuit ) and state[162] > 0:
                    returnAction = 52
        else:
            returnAction == -1
            myCards = np.where(state[0:52] == 1)[0]
            numCards = np.zeros(13)
            for i in myCards:
                numCards[i%13] +=1
            numCards3 = np.where(numCards > 2)[0]
            if len(numCards3) > 0:
                if numCards3[0] <= 4: 
                    for j in range(4):
                        if j*13 + numCards3[0] in ValidAction and j!= trumpSuit:
                            returnAction = j*13 + numCards3[0]
                            break

            
            if returnAction == -1:  
                numCards2 = np.where(numCards >= 2)[0]
                if len(numCards2) > 0:
                    if numCards2[0] <= 4: 
                        for j in range(4):
                            if j*13 + numCards2[0] in ValidAction and j != trumpSuit:
                                returnAction = j*13 + numCards2[0]
                                break
            

            

            if returnAction == -1:
                for i in range(13):
                    for j in range(4):
                        if i + 13*j in ValidAction and j != trumpSuit:
                              returnAction = i + 13* j
                              break
                    if returnAction != -1:
                        break

        # if np.sum(state[0:52]) <= 4  and state[162] == 0:
        #     if len(ValidAction) > 2:
        #         returnAction = ValidAction[1]

    if returnAction == -1:
        returnAction = ValidAction[0]
    
    # if per[0][1] >= np.sum(state[52:104]) and per[0][0] == 1:
    #     print('swap 1')
    #     per[0][0] = 0              

    # if per[0][1] >= np.sum(state[52:104]) and per[0][0] == 10:
    #     print('swap 2')
    #     per[0][0] = 1

    # if per[0][0] == 1:
    #     if returnAction != 52:
    #             if returnAction % 13 > 6  and state[162] > 0 and 52 in ValidAction:
    #                 returnAction = 52


    # if state[157] == 1 and returnAction == 52:
    #     per[0][0] = 1
    # if state[156] == 1 and np.sum(state[52:104]) == 0:
    #     per[0][0] = 10

    # per[0][1] = np.sum(state[52:104])

    # print('My cards:',np.where(state[0:52] == 1)[0])
    # print('Defend card:',np.where(state[52:104] == 1)[0])
    # print('Attack card:',np.where(state[104:156] == 1)[0])
    # print('Attack - defend:',state[156:158])
    # print('Trump suit:',state[158:162], trumpSuit)
    # print('Left Card: Deck and three rival:',state[162:166])
    # print(ValidAction, returnAction)
    # print('----------------')

    return returnAction, per

In [30]:
for i in range(10):
    win, per = numba_main_2(SuperStupidBot,10000,per,1)
    print(i, win)

0 5220
1 5132
2 5194
3 5152
4 5181
5 5125
6 5189
7 5034
8 5222
9 5199
